In [1]:
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
from sklearn.model_selection import train_test_split

from ipynb.fs.full.features import features_independientes_precio, features_dependientes_precio

df_train = pd.read_csv('./data/train.csv')

# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

In [10]:
def RMSLE(actual, pred):
    return (np.mean((np.log(actual + 1) - np.log(pred + 1)) ** 2)) **.5

In [20]:
def llenar_nulls(df):
    df['metrostotales'] = df['metrostotales'].fillna(df['metroscubiertos'])
    df['metroscubiertos'] = df['metrostotales'].fillna(df['metrostotales'])
    
    df['habitaciones'] = df['habitaciones'].fillna(df['habitaciones'].mean())
    df['garages'] = df['garages'].fillna(df['garages'].mean())
    df['banos'] = df['banos'].fillna(df['banos'].mean())

df_train_f = features_independientes_precio(df_train)
df_train_f = features_dependientes_precio(df_train_f, df_train)

df_test_f = features_independientes_precio(df_test)
df_test_f = features_dependientes_precio(df_test_f, df_train)

llenar_nulls(df_train_f)
llenar_nulls(df_test_f)

In [31]:
from sklearn.neural_network import MLPRegressor


df_nn = df_train_f.copy().drop(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad', 'provincia',
                                'fecha', 'zona', 'intervalo_metros_totales', 'intervalo_metros_cubiertos', 'id', 
                               'antiguedad', 'idzona', 'lat', 'lng', 'porcentaje_metros',
                               'promedio_metros_tipo_propiedad'], axis=1)

df_nn = df_nn[['habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'escomercial', 'promedio_precio_ciudad',
       'promedio_id_zona', 'precio']]

df_nn_test, df_nn_train = train_test_split(df_nn, test_size=0.25, random_state=1)

y_test = df_nn_test['precio']
y_train = df_nn_train['precio']
x_test = df_nn_test.drop('precio', axis=1)
x_train = df_nn_train.drop('precio', axis=1)

mlp = MLPRegressor(hidden_layer_sizes=(30, 30, 30, 30, 30), verbose=True)
mlp.fit(x_train,y_train)

mlpr_rmsle_train = RMSLE(y_train, mlp.predict(x_train))
mlpr_rmsle = RMSLE(y_test, mlp.predict(x_test))
print(f"RMSLE MLPRegressor (train): {mlpr_rmsle_train:.5f}")
print(f"RMSLE MLPRegressor: {mlpr_rmsle:.5f}")

Iteration 1, loss = 1394787890012.46240234
Iteration 2, loss = 1042875947973.60559082
Iteration 3, loss = 1043218417521.38354492
Iteration 4, loss = 1043653177685.19995117
Iteration 5, loss = 1044120924909.98864746
Iteration 6, loss = 1044132027680.98254395
Iteration 7, loss = 1045604746821.99597168
Iteration 8, loss = 1044935254940.94201660
Iteration 9, loss = 1043172127122.99963379
Iteration 10, loss = 1043150523005.39855957
Iteration 11, loss = 1041603234745.77294922
Iteration 12, loss = 1042833573637.39331055
Iteration 13, loss = 1042835949078.56176758
Iteration 14, loss = 1043155947869.90747070
Iteration 15, loss = 1043366285511.87841797
Iteration 16, loss = 1043294644818.34509277
Iteration 17, loss = 1041606988886.75573730
Iteration 18, loss = 1042265198263.03088379
Iteration 19, loss = 1042474952868.70849609
Iteration 20, loss = 1043533114578.13525391
Iteration 21, loss = 1042577328338.64892578
Iteration 22, loss = 1041352084930.19775391
Iteration 23, loss = 1042111037285.385986